In [1]:
# 10 songs for each of 4 trials for each of 31 subjects = total of 1240 playings
# 12 target classes: HAPPY, LOW VALENCE, LOW TENSION, LOW ENERGY, HIGH ENERGY, SURPRISE, FEAR, TENDER, ANGER, SAD, HIGH VALENCE, HIGH TENSION
# ? Unknown how many playings fall under each target

In [2]:
import mne # package for reading edf data
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
from scipy.signal import butter, sosfiltfilt, sosfreqz  

from data_tools import * # i made all the functions into a python file

In [3]:
df = get_recording_events(2,3) # you can now run the functions here
df.head() # gets the head of the dataframe

Extracting EDF parameters from c:\Users\Staro\Documents\GitHub\COGS189-final-project\data\sub-02\eeg\sub-02_task-run3_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
c:\Users\Staro\Documents\GitHub\COGS189-final-project\data_tools.py:17: RuntimeWarning: Invalid date encountered (2014-00-00 00:00:00).
  data = mne.io.read_raw_edf(file)


,time,song_clip,FP1,FP2,F7,F3,Fz,F4,F8,T3,...,T4,T5,P3,Pz,P4,T6,O1,O2,Number,TARGET
0,6703,29,-149.623601,-111.775574,49.314735,-1.279259,-65.707514,-49.011326,-12.416786,10.371734,...,9.759761,-35.782211,-16.815982,8.581891,72.525610,49.671773,102.469989,-2.850259,29,HAPPY
1,6704,29,-173.877896,-116.794177,33.418188,-1.556187,-69.779909,-51.001599,-13.867926,31.866286,...,12.467998,-16.045504,-4.362878,12.894548,76.087524,50.927955,112.681901,2.977428,29,HAPPY
2,6705,29,-175.805390,-118.389954,64.473467,-7.593208,-69.412832,-52.480088,-16.696420,13.820617,...,9.736810,-9.136154,3.951592,12.663714,72.600796,39.190897,109.743665,-2.839427,29,HAPPY
3,6706,29,-170.906344,-119.754459,54.479436,-3.910071,-68.782323,-53.006089,-20.090613,-8.301004,...,4.871165,-7.288654,6.899788,14.493496,71.651578,32.960740,111.808649,-2.427806,29,HAPPY
4,6707,29,-162.594027,-116.215997,87.748158,-12.411747,-72.142157,-50.020679,-13.720352,21.206103,...,10.517150,-4.494874,3.421475,13.761583,74.085710,37.845894,114.674970,1.200959,29,HAPPY


this is an example of what will be in the labels list

In [4]:
df.groupby('song_clip').head(1).TARGET.to_list()

['HAPPY',
 'FEAR',
 'ANGER',
 'SAD',
 'SURPRISE',
 'HIGH VALENCE',
 'SAD',
 'HIGH VALENCE',
 'SAD',
 'SURPRISE']

This is an example of what will be in the lists (one item in the recordings list). If you wish to keep the time column, remove it from the drop columns list
it actually isnt always 16000 columns, sometimes for whatever reason the last one of 10 cuts off early for reasons unknown.

In [5]:

# df.song_clip.unique()
df[df.song_clip==29].drop(columns=['song_clip','Number','TARGET','time'])

,FP1,FP2,F7,F3,Fz,F4,F8,T3,C3,Cz,C4,T4,T5,P3,Pz,P4,T6,O1,O2
0,-149.623601,-111.775574,49.314735,-1.279259,-65.707514,-49.011326,-12.416786,10.371734,-32.831533,-28.469000,-12.807568,9.759761,-35.782211,-16.815982,8.581891,72.525610,49.671773,102.469989,-2.850259
1,-173.877896,-116.794177,33.418188,-1.556187,-69.779909,-51.001599,-13.867926,31.866286,-24.871664,-25.202006,-11.848305,12.467998,-16.045504,-4.362878,12.894548,76.087524,50.927955,112.681901,2.977428
2,-175.805390,-118.389954,64.473467,-7.593208,-69.412832,-52.480088,-16.696420,13.820617,-5.290389,-25.539442,-15.896393,9.736810,-9.136154,3.951592,12.663714,72.600796,39.190897,109.743665,-2.839427
3,-170.906344,-119.754459,54.479436,-3.910071,-68.782323,-53.006089,-20.090613,-8.301004,1.594897,-25.505698,-18.966032,4.871165,-7.288654,6.899788,14.493496,71.651578,32.960740,111.808649,-2.427806
4,-162.594027,-116.215997,87.748158,-12.411747,-72.142157,-50.020679,-13.720352,21.206103,-17.349589,-25.766444,-14.610981,10.517150,-4.494874,3.421475,13.761583,74.085710,37.845894,114.674970,1.200959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15996,-246.118752,-71.002319,-68.266054,-88.566823,-9.384956,-2.481576,-35.143119,-33.627648,-80.909136,-21.091422,-37.000165,17.976275,-37.900077,-38.746095,7.382679,61.454542,-5.194215,76.097780,-10.216109
15997,-243.147199,-67.764510,-65.784314,-91.696104,-7.847552,0.247942,-32.708155,-32.547695,-83.615492,-22.468774,-38.189650,15.956573,-39.056642,-39.899332,7.804937,62.206397,-1.501800,74.001975,-4.648393
15998,-236.400971,-61.473912,-61.089132,-105.459403,-3.969492,1.484755,-28.600690,-30.736161,-84.610475,-21.839916,-37.901871,11.595854,-35.586947,-36.662827,7.652924,63.578533,-2.757982,86.124767,2.825779
15999,-238.248153,-61.867074,-58.808614,-96.902342,-4.798654,2.167134,-27.493888,-31.676765,-80.391745,-20.373604,-39.264024,14.900820,-33.499122,-33.435622,10.715699,64.264601,-2.757982,92.422455,3.248232


In [6]:
fs = 240.0                      # Hz; sampling rate
dt = 1000/fs                    # ms; time between samples
sdt = np.round(dt).astype(int); # rounded dt so that we can index samples
hp = 0.1                        # Hz; our low cut for our bandpass
lp = 30.                        # Hz; our high cut for our bandpass
order = 3                       # filter order (functionally doubled)

# Create our filter coefficient as as a second-order section
# Note: by defining 'fs' we don't divide our windows by the Nyquist
sos = butter(order, [hp, lp], analog = False, btype = 'band', output = 'sos', fs = fs)

In [23]:
# Create empty lists to store our data. We'll conver them into np.arrays at the end
train_data = []
train_labels = []
test_data = []
test_labels = []

test_set = np.random.choice(124, size=24, replace=False)

# Going through all 124 trials
for subject in range(1,32):
    for trial in range(2,6):
        # fileName = f"data/sub-{format(subject, '02x')}/eeg/sub-{format(subject, '02x')}_task-run{trial}_eeg.edf"
        # print('Processing: ' + fileName)
        # data = mne.io.read_raw_edf(fileName)
        # trialRecording = data.to_data_frame().infer_objects()
        df = get_recording_events(subject, trial)
        labels = df.groupby('song_clip').head(1).TARGET.to_list() # Should be an array of 10 labels for each of the songs played in this trial
        recordings = [df[df.song_clip==x].drop(columns=['song_clip','Number','TARGET','time']) for x in df.song_clip.unique()] # Should be a 3D array of 10 recordings, each with 16000 rows and 20 columns NOTE the row counts are inconsistent and there are 19 columns
        # Bens code to get individual playings




        # Going through all 10 playings
        for playing in range(0,10):
            recording = recordings[playing]
            if recording.shape[0] < 16000:
                continue
            recording = recording.iloc[:16000,:]

            # Epoch and correct DC offset of signal pre-filtering
            # recording = recording[e_s+t0:t0+e_e, :] - np.mean(recording_[e_s+t0:t0+e_e, :], 0)
            # recording = sosfiltfilt(sos, recording)
            
            # Now let's baseline correct
            # data = data - np.mean(data[bl_s+np.abs(e_s):np.abs(e_s)+bl_e, :], 0)
            
            # Append data to correct locations
            if subject == 31:
                test_data.append(recording)
                test_labels.append(labels[playing]) # target or non-target

            else:
                train_data.append(recording)
                train_labels.append(labels[playing]) # target or non-target
                    
print('Processing completed!')

# Convert all of our lists into numpy arrays
# train_data = np.array(train_data)
# train_labels = np.array(train_labels)
# test_data = np.array(test_data)
# test_labels = np.array(test_labels)
test_labels

Extracting EDF parameters from c:\Users\Staro\Documents\GitHub\COGS189-final-project\data\sub-01\eeg\sub-01_task-run2_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
c:\Users\Staro\Documents\GitHub\COGS189-final-project\data_tools.py:17: RuntimeWarning: Invalid date encountered (2014-00-00 00:00:00).
  data = mne.io.read_raw_edf(file)
Extracting EDF parameters from c:\Users\Staro\Documents\GitHub\COGS189-final-project\data\sub-01\eeg\sub-01_task-run3_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
c:\Users\Staro\Documents\GitHub\COGS189-final-project\data_tools.py:17: RuntimeWarning: Invalid date encountered (2014-00-00 00:00:00).
  data = mne.io.read_raw_edf(file)
Extracting EDF parameters from c:\Users\Staro\Documents\GitHub\COGS189-final-project\data\sub-01\eeg\sub-01_task-run4_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
c:\Users\Staro\Documen

array([], dtype=float64)

In [24]:
lst = [x.shape[0] for x in train_data]
pd.value_counts(lst).sort_index(ascending=False)

16000    79
dtype: int64

In [22]:
lst = [x.shape[0] for x in train_data]
pd.value_counts(lst).sort_index(ascending=False)

16002      10
16001    1154
16000       7
15486       1
15252       1
15000       1
14962       1
14086       1
13403       1
13272       1
13140       1
12408       1
12001       1
11507       1
11090       1
10725       1
10541       1
10486       1
10452       1
10436       1
10420       1
10407       1
10290       1
10234       1
10097       1
9834        1
9573        1
9526        1
9520        1
9320        1
8765        1
8548        1
dtype: int64

In [ ]:
# Seperate target and non-target for plotting
tar     = train_data[np.where(train_labels == 1)[0], :, :]
non_tar = train_data[np.where(train_labels == 0)[0], :, :]

print('We have %d target trials' % tar.shape[0])
print('We have %d non-target trials' % non_tar.shape[0])

# We'll take the average of all trials to create an averaged ERP
tar_avg     = np.mean(tar, 0)
non_tar_avg = np.mean(non_tar, 0)

# Define channel of interest and create an array of time points
chan = 'Cz' # let's plot Cz
ch = np.where(channels == chan)[0][0]
times = np.linspace(epoch_start, epoch_end, train_data.shape[1])

# Initialize plot and calculate min and max y value
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(20, 6))
min_y = min(np.min(tar_avg), np.min(non_tar_avg))
max_y = max(np.max(tar_avg), np.max(non_tar_avg))

# Plot x and y axes
plt.plot([np.min(times), np.max(times)], [0, 0], color='k');  # x-axis
plt.plot([0, 0], [min_y, max_y], color='k');                  # y-axis

# Plot our averaged ERPs
plt.plot(times, tar_avg[:, ch], 'b', linewidth=4)
plt.plot(times, non_tar_avg[:, ch], 'r', linewidth=4)

# Highlight the baseline window and window of interest of our ERP
baseline = patches.Rectangle([baseline_start, min_y], baseline_end, np.abs(min_y)+max_y, 
                             color='c', alpha=0.2)
erp_win = patches.Rectangle([erp_start, min_y], erp_end-erp_start, np.abs(min_y)+max_y, 
                             color='lime', alpha=0.3)

# Add our baseline and window of interest highlights
ax.add_patch(baseline)
ax.add_patch(erp_win)

# Manually create legends since patches will corrupt default handles
legend_ = [patches.Patch(color='b', label = 'Target (oddball)'),
           patches.Patch(color='r', label = 'Non-target (standard)')]

# Finalize plot and set a high DPI for a crisp, hi-res figure
plt.xlabel('Time (msec)');
plt.ylabel('Amplitude (A/D Units)');
plt.legend(handles=legend_, loc="upper right");
plt.title('Event Related Potentials at channel %s' % chan);
fig.set_dpi(216);
plt.show();

In [ ]:
# Let's compute the windowed means within erp_start and erp_end
num_points = 5; # we will divide our window into num_points means

# Define a simple windowed means function
def wm(x, start, end, num_points):
    num_trials = x.shape[0] # assumes first dem is numb observations
    w = np.round((start+end)/num_points).astype(int)
    y = np.zeros((num_points, x.shape[-1], num_trials)) # assumes num chans as last dimension
    for i in range(0, num_points):
        s = start + (w * i)
        e = end   + (w * i)
        y[i, :, :] = np.mean(x[:, s:e, :], 1).T
    return y

# Combine into a single train variable. Also create labels
X_train    = wm(train_data, erp_s, erp_e, num_points)
markers_train = np.vstack((train_labels, train_markers)).T
y = train_labels

# Now let's compute windowed means of our test data
X_test = wm(test_data, erp_s, erp_e, num_points)
markers_test = test_markers

# Let's print out the shapes of our data
print('X_train shape is: ' + str(X_train.shape))
print('y shape is......: ' + str(y.shape))
print('X_test shape is.: ' + str(X_test.shape))

In [ ]:
# Since our X is 3D, we must flatten our data. We will then transpose it for sklearn
X_train = X_train.reshape(-1, X_train.shape[-1]).T
X_test = X_test.reshape(-1, X_test.shape[-1]).T

# Let's print out the new shape
print('X_train shape is now: ' + str(X_train.shape))
print('X_test  shape is now: ' + str(X_test.shape))

In [ ]:
# Train our classifier (this may take a while via JupyterHub)
clf_lsqrs = LinearDiscriminantAnalysis(solver = 'lsqr',  shrinkage = 'auto').fit(X_train, y)

In [ ]:
# Let's do 5-fold cross validation
score_lsqrs = cross_val_score(clf_lsqrs.fit(X_train, y), X_train, y, cv = 5)

# We will print out the mean score
print("solver = lsqr  accuracy: %f" % np.mean(score_lsqrs))